# Название проекта

Анализ базы данных сервиса для чтения книг по подписке (SQL)

# Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Задачи проекта

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


# Описание данных

Таблица "books" содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица "authors" содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

Таблица "publishers" содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица "ratings" содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица "reviews" содержит данные о пользовательских обзорах на книги:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

# Исследование таблиц
Сначала нам надо импортировать библиотеки:

In [9]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

А теперь зададим параметры конфига:

In [10]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Запрос к таблице "books"

In [11]:
query = '''

SELECT * 
FROM books
LIMIT 5

'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Запрос к таблице "authors"

In [12]:
query = '''

SELECT * 
FROM authors
LIMIT 5

'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


## Запрос к таблице "publishers"

In [13]:
query = '''

SELECT * 
FROM publishers
LIMIT 5

'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Запрос к таблице "ratings"

In [14]:
query = '''

SELECT * 
FROM ratings
LIMIT 5

'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


##  Запрос к таблице "reviews"

In [15]:
query = '''

SELECT * 
FROM reviews
LIMIT 5

'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Исследование

## Задача 1

Посчитайте, сколько книг вышло после 1 января 2000 года

In [16]:
query = '''

SELECT COUNT(DISTINCT book_id)  
FROM books
WHERE publication_date > '2000-01-01'

'''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


## Задача 2

Для каждой книги посчитайте количество обзоров и среднюю оценку

In [23]:
query = '''

SELECT b.title, COUNT(DISTINCT rv.review_id) as review_count, AVG(rt.rating) as reating_mean  
FROM books as b
LEFT OUTER JOIN ratings AS rt ON b.book_id = rt.book_id
LEFT OUTER JOIN reviews AS rv ON b.book_id = rv.book_id
GROUP BY b.book_id
ORDER BY review_count desc
'''
pd.io.sql.read_sql(query, con = engine)

,title,review_count,reating_mean
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


## Задача 3

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [18]:
query = '''

SELECT pb.publisher, COUNT(b.title)  
FROM books as b
LEFT OUTER JOIN publishers AS pb ON b.publisher_id = pb.publisher_id
WHERE b.num_pages > 50 
GROUP BY pb.publisher
ORDER BY COUNT(b.title) desc
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


## Задача 4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [27]:
query = '''

SELECT author, AVG(sub.avg_rating) AS reating_mean
FROM (SELECT author, books.book_id, AVG(rating) AS avg_rating
FROM authors
LEFT JOIN books ON books.author_id = authors.author_id
LEFT JOIN ratings ON ratings.book_id = books.book_id
GROUP BY author, books.book_id
HAVING COUNT(rating) > 50) AS sub
GROUP BY author
ORDER BY reating_mean DESC
LIMIT 1

'''
pd.io.sql.read_sql(query, con = engine)

,author,reating_mean
0,J.K. Rowling/Mary GrandPré,4.283844


## Задача 5

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [20]:
query = '''

SELECT ROUND(AVG(review_count)) as mean_review_count
FROM (SELECT COUNT(review_id) as review_count
FROM reviews
WHERE username IN (SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(ratings) > 50)
GROUP BY username) as sub_review_count 

'''
pd.io.sql.read_sql(query, con = engine)

,mean_review_count
0,24.0


# Вывод исследования

Мы провели исследование в ходе которого у знали что:
- после 1 января 2000 года вышло 819 книг;
- На книгу Twilight (Twilight #1) вышло максимальное количество обзоров - 7, рейтинг книги - 3.7;
- издательство, выпустившее наибольшее число книг толще 50 страниц: Penguin Books, они выпустили 42 книги;
- автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré с рейтингом 4.28;
- среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24.